In [1]:
from Translator import translate_line
import nltk
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
import rdflib
import re
from Word2Vec import Word2Vec_Evaluation
import sys
from scipy.stats import spearmanr
import requests
import numpy as np
import enchant
import bs4
import requests
from collections import defaultdict
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [2]:
words = ['brasil', 'registrou', 'reducao', 'numero', 'deteccoes', 'aids', 'segundo', 'boletim', 'epidemiologico', 'divulgado', 'nesta', 'terca', 'feira', 'saude', 'medico', 'hospital', 'medicos', 'doenca', 'casos', 'aids', 'virus', 'tratamento', 'hospitais']
words = list(set(words))
print(words)

['reducao', 'casos', 'virus', 'brasil', 'tratamento', 'doenca', 'numero', 'nesta', 'hospital', 'segundo', 'aids', 'hospitais', 'feira', 'medico', 'terca', 'epidemiologico', 'deteccoes', 'divulgado', 'boletim', 'saude', 'registrou', 'medicos']


### Rank words based on Word2Vec centroid

In [3]:
w2v = Word2Vec_Evaluation(model_path="w2vmodels/STM_CBOW_65dim", stem=True)
ranked = w2v.rank_topic(" ".join(words))
words,rank = zip(*ranked)
print("ranked words: "," ".join(words))

ranked words:  saude hospitais medicos doenca tratamento epidemiologico registrou aids brasil casos hospital nesta segundo numero reducao divulgado virus feira terca boletim deteccoes


### Translate words

In [4]:
en_words = translate_line(" ".join(words), "pt-br", "en_US")
en_words = tokenizer.tokenize(en_words.lower())
d = enchant.Dict("en_US")
only_english=[]
for word in en_words:
    if(d.check(word)):
        only_english.append(word)
en_words = only_english
print(en_words)

['health', 'hospitals', 'medical', 'treatment', 'epidemiological', 'treatment', 'registered', 'aids', 'cases', 'hospital', 'in', 'this', 'second', 'number', 'reduction', 'publicized', 'virus', 'bulletin', 'detections']


### Extract nouns

In [7]:
en_nouns=[]
for word,pos in nltk.pos_tag(en_words):
    if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
        en_nouns.append(word)
print(en_nouns) 
en_words = en_nouns

['health', 'hospitals', 'treatment', 'treatment', 'aids', 'hospital', 'cases', 'number', 'reduction', 'virus', 'bulletin', 'detections']


### Removing Stopwords

In [5]:
stop_words = set(stopwords.words('english')) 
final_words = [w for w in en_words if not w in stop_words]
print(final_words)

['health', 'hospitals', 'medical', 'treatment', 'epidemiological', 'treatment', 'registered', 'aids', 'cases', 'hospital', 'second', 'number', 'reduction', 'publicized', 'virus', 'bulletin', 'detections']


In [40]:
print("TOP 5 words: "+" ".join(final_words[:5]))

TOP 5 words: health hospitals medical treatment epidemiological


In [41]:
found_onto = defaultdict(list)
word = "+".join(final_words[:5])
print("looking for:", word)
r = requests.get(r"http://swoogle.umbc.edu/2006/index.php?option=com_frontpage&service=search&queryType=search_swd_ontology&searchString={}&searchStart=1".format(word))
soup = bs4.BeautifulSoup(r.text)
links = soup.findAll('a')
for link in links:
    if("class" in dict(link.attrs).keys() and link.attrs['class'][0]=='external'):
        found_onto[word].append(link.attrs['href'])

word = "+".join(final_words[:4])
print("looking for:", word)
r = requests.get(r"http://swoogle.umbc.edu/2006/index.php?option=com_frontpage&service=search&queryType=search_swd_ontology&searchString={}&searchStart=1".format(word))
soup = bs4.BeautifulSoup(r.text)
links = soup.findAll('a')
for link in links:
    if("class" in dict(link.attrs).keys() and link.attrs['class'][0]=='external'):
        found_onto[word].append(link.attrs['href'])

word = "+".join(final_words[1:5])
print("looking for:", word)
r = requests.get(r"http://swoogle.umbc.edu/2006/index.php?option=com_frontpage&service=search&queryType=search_swd_ontology&searchString={}&searchStart=1".format(word))
soup = bs4.BeautifulSoup(r.text)
links = soup.findAll('a')
for link in links:
    if("class" in dict(link.attrs).keys() and link.attrs['class'][0]=='external'):
        found_onto[word].append(link.attrs['href'])

word = "+".join(final_words[:3])
print("looking for:", word)
r = requests.get(r"http://swoogle.umbc.edu/2006/index.php?option=com_frontpage&service=search&queryType=search_swd_ontology&searchString={}&searchStart=1".format(word))
soup = bs4.BeautifulSoup(r.text)
links = soup.findAll('a')
for link in links:
    if("class" in dict(link.attrs).keys() and link.attrs['class'][0]=='external'):
        found_onto[word].append(link.attrs['href'])

word = "+".join(final_words[1:4])
print("looking for:", word)
r = requests.get(r"http://swoogle.umbc.edu/2006/index.php?option=com_frontpage&service=search&queryType=search_swd_ontology&searchString={}&searchStart=1".format(word))
soup = bs4.BeautifulSoup(r.text)
links = soup.findAll('a')
for link in links:
    if("class" in dict(link.attrs).keys() and link.attrs['class'][0]=='external'):
        found_onto[word].append(link.attrs['href'])

word = "+".join(final_words[2:5])
print("looking for:", word)
r = requests.get(r"http://swoogle.umbc.edu/2006/index.php?option=com_frontpage&service=search&queryType=search_swd_ontology&searchString={}&searchStart=1".format(word))
soup = bs4.BeautifulSoup(r.text)
links = soup.findAll('a')
for link in links:
    if("class" in dict(link.attrs).keys() and link.attrs['class'][0]=='external'):
        found_onto[word].append(link.attrs['href'])
        
word = "+".join(final_words[:3])
print("looking for:", word)
r = requests.get(r"http://swoogle.umbc.edu/2006/index.php?option=com_frontpage&service=search&queryType=search_swd_ontology&searchString={}&searchStart=1".format(word))
soup = bs4.BeautifulSoup(r.text)
links = soup.findAll('a')
for link in links:
    if("class" in dict(link.attrs).keys() and link.attrs['class'][0]=='external'):
        found_onto[word].append(link.attrs['href'])

word = "+".join(final_words[:2])
print("looking for:", word)
r = requests.get(r"http://swoogle.umbc.edu/2006/index.php?option=com_frontpage&service=search&queryType=search_swd_ontology&searchString={}&searchStart=1".format(word))
soup = bs4.BeautifulSoup(r.text)
links = soup.findAll('a')
for link in links:
    if("class" in dict(link.attrs).keys() and link.attrs['class'][0]=='external'):
        found_onto[word].append(link.attrs['href'])

word = "+".join(final_words[1:3])
print("looking for:", word)
r = requests.get(r"http://swoogle.umbc.edu/2006/index.php?option=com_frontpage&service=search&queryType=search_swd_ontology&searchString={}&searchStart=1".format(word))
soup = bs4.BeautifulSoup(r.text)
links = soup.findAll('a')
for link in links:
    if("class" in dict(link.attrs).keys() and link.attrs['class'][0]=='external'):
        found_onto[word].append(link.attrs['href'])

word = "+".join(final_words[2:4])
print("looking for:", word)
r = requests.get(r"http://swoogle.umbc.edu/2006/index.php?option=com_frontpage&service=search&queryType=search_swd_ontology&searchString={}&searchStart=1".format(word))
soup = bs4.BeautifulSoup(r.text)
links = soup.findAll('a')
for link in links:
    if("class" in dict(link.attrs).keys() and link.attrs['class'][0]=='external'):
        found_onto[word].append(link.attrs['href'])

word = "+".join(final_words[3:5])
print("looking for:", word)
r = requests.get(r"http://swoogle.umbc.edu/2006/index.php?option=com_frontpage&service=search&queryType=search_swd_ontology&searchString={}&searchStart=1".format(word))
soup = bs4.BeautifulSoup(r.text)
links = soup.findAll('a')
for link in links:
    if("class" in dict(link.attrs).keys() and link.attrs['class'][0]=='external'):
        found_onto[word].append(link.attrs['href'])

looking for: health+hospitals+medical+treatment+epidemiological
looking for: health+hospitals+medical+treatment
looking for: hospitals+medical+treatment+epidemiological
looking for: health+hospitals+medical
looking for: hospitals+medical+treatment
looking for: medical+treatment+epidemiological
looking for: health+hospitals+medical
looking for: health+hospitals
looking for: hospitals+medical
looking for: medical+treatment
looking for: treatment+epidemiological


In [42]:
ontos = defaultdict()
for on in found_onto.keys():
    if(len(found_onto[on])>0):
        ontos[found_onto[on][0]]=0
    if(len(found_onto[on])>1):
        ontos[found_onto[on][0]]=0

for on in found_onto.keys():
    if(len(found_onto[on])>0):
        ontos[found_onto[on][0]]+=1
    if(len(found_onto[on])>1):
        ontos[found_onto[on][0]]+=1
ontos = list(ontos.items())
ontos.sort(key = lambda x: x[1], reverse=True)
print(ontos[0][0])
print(ontos[1][0])

http://swpatho.ag-nbi.de/context/naics.owl
http://www.mindswap.org/2004/multipleOnt/FactoredOntologies/NCI/Ontology19.owl


In [27]:
onto_URLs = {}
onto_URLs['sport'] = "https://www.bbc.co.uk/ontologies/sport/3.2.ttl" 
onto_URLs['wildlife'] = "https://www.bbc.co.uk/ontologies/wo/1.1.ttl"
onto_URLs['storyline'] = "https://www.bbc.co.uk/ontologies/storyline/0.3.ttl"
onto_URLs['provenance'] = "https://www.bbc.co.uk/ontologies/provenance/1.9.ttl"
onto_URLs['programmes'] = "https://www.bbc.co.uk/ontologies/po/1.1.ttl"
#onto_URLs['politics'] = "https://www.bbc.co.uk/ontologies/politics/0.9.ttl"
onto_URLs['journalism'] = "https://www.bbc.co.uk/ontologies/journalism/0.2.ttl"
onto_URLs['food'] = "https://www.bbc.co.uk/ontologies/fo/1.1.ttl"
onto_URLs['curriculum'] = "https://www.bbc.co.uk/ontologies/curriculum/1.3.ttl"
onto_URLs['creativework'] = "https://www.bbc.co.uk/ontologies/creativework/1.11.ttl"
#onto_URLs['coreconcepts'] = "https://www.bbc.co.uk/ontologies/coreconcepts/1.13.ttl"
onto_URLs['cms'] = "https://www.bbc.co.uk/ontologies/cms/3.7.ttl"
onto_URLs['business'] = "https://www.bbc.co.uk/ontologies/business/0.5.ttl"

In [ ]:
for onto in onto_URLs.keys():
    print("For",onto,"ontology:")
    g = rdflib.Graph()
    g.parse(onto_URLs[onto])
    hits = 0
    for word in final_words:
        for i in g.all_nodes():
            if(type(i)==rdflib.term.URIRef):
                if(re.search(word, i, re.IGNORECASE)!= None):
                    hits+=1
                    print(i,"MATCHED WITH:",word)
            if(type(i)==rdflib.term.Literal):
                 if(re.search(word, i, re.IGNORECASE)!= None):
                     hits+=1
                     print(i,"MATCHED WITH:",word)
    print("{} hits!".format(hits))
    print()

In [70]:
# wnl = WordNetLemmatizer()
# lemmatized = [wnl.lemmatize(i) for i in en_words]
# lemmatized.extend(en_words)
# lemmatized = list(set(lemmatized))
# print(lemmatized)